In [17]:
import numpy as np
import gensim
import nltk
import re
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
import seaborn as sns
import pandas as pd
from nltk.tokenize import sent_tokenize
import multiprocessing
import os
from pathlib import Path
import io
from gensim.models import Phrases
from textblob import TextBlob
from nltk.tag import pos_tag
from nltk.tokenize import word_tokenize
from nltk.tokenize import RegexpTokenizer
from pandas import DataFrame

import torch
from torch.utils.data import Dataset, DataLoader
from pytorch_transformers import BertTokenizer, BertForSequenceClassification, BertConfig
from torch.optim import Adam
import torch.nn.functional as F
import time

In [18]:
def character_all_section(text):


    # Number of Characters
    def NumberofCharacters(text):

        essay_input_corpus = str(text) #문장입력
        essay_input_corpus = essay_input_corpus.lower()#소문자 변환

        sentences  = sent_tokenize(essay_input_corpus) #문장 토큰화
        total_sentences = len(sentences)#토큰으로 처리된 총 문장 수
        total_words = len(word_tokenize(essay_input_corpus))# 총 단어수
        split_sentences = []
        for sentence in sentences:
            processed = re.sub("[^a-zA-Z]"," ", sentence)
            words = processed.split()
            split_sentences.append(words)

        skip_gram = 1
        workers = multiprocessing.cpu_count()
        bigram_transformer = Phrases(split_sentences)

        model = gensim.models.word2vec.Word2Vec(bigram_transformer[split_sentences], workers=workers, sg=skip_gram, min_count=1)

        model.train(split_sentences, total_examples=sum([len(sentence) for sentence in sentences]), epochs=100)
        
        #모델 설계 완료

        #캐릭터 표현하는 단어들을 리스트에 넣어서 필터로 만들고
        character_list = ['i', 'my', 'me', 'mine', 'you', 'your', 'they','them',
                        'yours', 'he','him','his' 'she','her','it','someone','their', 'myself', 'aunt',
                        'brother','cousin','daughter','father','grandchild','granddaughter','granddson','grandfather',
                        'grandmother','great-grandchild','husband','ex-husband','son-in-law', 'daughter-in-law','mother',
                        'niece','nephew','parents','sister','son','stepfather','stepmother','stepdaughter', 'stepson',
                        'twin','uncle','widow','widower','wife','ex-wife','aunt',
                        'baby', 'beget', 'brother', 'buddy', 'conserve', 'counterpart', 'cousin', 'daughter', 'don', 'duplicate', 'ex',
                        'father', 'forefather', 'founder', 'gemini', 'grandchild', 'granddaughter', 'grandfather', 'grandma', 'he', 'helium',
                        'husband', 'i', 'in', 'iodine', 'law', 'maine', 'match', 'mine', 'mother', 'nephew', 'niece', 'one', 'parent', 'person',
                        'rear', 'sister', 'son', 'stepdaughter', 'stepfather', 'stepmother', 'stepson', 'twin', 'uncle', 'widow', 'widower', 'wife']
        
        ####문장에 char_list의 단어들이 있는지 확인하고, 있다면 유사단어를 추출한다.
        #우선 토큰화한다.
        retokenize = RegexpTokenizer("[\w]+") #줄바꿈 제거하여 한줄로 만들고
        token_input_text = retokenize.tokenize(essay_input_corpus)
        #print (token_input_text) #토큰화 처리 확인.. 토큰들이 리스트에 담김
        #리트스로 정리된 개별 토큰을 char_list와 비교해서 존재하는 것만 추출한다.
        filtered_chr_text = []
        for k in token_input_text:
            for j in character_list:
                if k == j:
                    filtered_chr_text.append(j)
        
        #print (filtered_chr_text) # 유사단어 비교 추출 완료, 겹치는 단어는 제거하자.
        
        filtered_chr_text_ = set(filtered_chr_text) #중복제거
        filtered_chr_text__ = list(filtered_chr_text_) #다시 리스트로 변환
        print (filtered_chr_text__) # 중복값 제거 확인
        
        for i in filtered_chr_text__:
            ext_sim_words_key = model.most_similar_cosmul(i) #모델적용
        
        char_total_count = len(filtered_chr_text) # 중복이 제거되지 않은 에세이 총 문장에 사용된 캐릭터 표현 수
        char_count_ = len(filtered_chr_text__) #중복제거된 캐릭터 표현 총 수
            
        result_char_ratio = round(char_total_count/total_words * 100, 2)
        return char_total_count



    # number_of_characters = NumberofCharacters(input_text) # 문장에서 키워드와 관련된 단어을 모두 추출하면 이런 결과가 나옴, 이 결과를 모두 합쳐서 캐릭터 총 값 계산해서 숫자로 출력
    # number_of_characters
    # print ('=============================================')
    # print ('Number of Characters :', number_of_characters)
    # print ('=============================================')

    ####################################
    #### Character Descriptiveness #####
    ####################################

    def character_descrip(text):

        input_sentence = text

        def findSentence(input_sentence):
            result = []

            data = str(input_sentence)
            #data = input_sentence.splitlines()
            
            findText = ['i', 'my', 'me', 'mine', 'you', 'your', 'they','them',
                        'yours', 'he','him','his' 'she','her','it','someone','their', 'myself', 'aunt',
                        'brother','cousin','daughter','father','grandchild','granddaughter','granddson','grandfather',
                        'grandmother','great-grandchild','husband','ex-husband','son-in-law', 'daughter-in-law','mother',
                        'niece','nephew','parents','sister','son','stepfather','stepmother','stepdaughter', 'stepson',
                        'twin','uncle','widow','widower','wife','ex-wife','aunt',
                        'baby', 'beget', 'brother', 'buddy', 'conserve', 'counterpart', 'cousin', 'daughter', 'don', 'duplicate', 'ex',
                        'father', 'forefather', 'founder', 'gemini', 'grandchild', 'granddaughter', 'grandfather', 'grandma', 'he', 'helium',
                        'husband', 'i', 'in', 'iodine', 'law', 'maine', 'match', 'mine', 'mother', 'nephew', 'niece', 'one', 'parent', 'person',
                        'rear', 'sister', 'son', 'stepdaughter', 'stepfather', 'stepmother', 'stepson', 'twin', 'uncle', 'widow', 'widower', 'wife']

            sentences = data.split(".")
            
            for sentence in sentences:
                for item in findText:
                    if item in sentence:
                        result.append(sentence)

            return result

        input_sent_included_character = findSentence(text) 
        input_sent_chr = set(input_sent_included_character) #중복값을 제거해보자
        input_sent_chr = '.'.join(input_sent_chr) #하나의 문자열로 합쳐야 원본 문장처럼 변환되고, 이것을 show/tell 분석코드에 넣게됨



        #입력된 전체 문장을 개별문장으로 분리하여 전처리 처리함
        def sentence_to_df(input_sentence):

            input_text_df = nltk.tokenize.sent_tokenize(input_sentence)
            test = []

            for i in range(0,len(input_text_df)):
                new_label = np.random.randint(0,2)  # 개별문장(input_text_df) 수만큼 0 또는 1 난수 생성
                data = [new_label, input_text_df[i]]
                test.append(data)

            #print(test)
            dataf = pd.DataFrame(test, columns=['label', 'text'])
            #print(dataf)
            return dataf


        class STDataset(Dataset):
            ''' Showing Telling Corpus Dataset '''
            def __init__(self, df):
                self.df = df

            def __len__(self):
                return len(self.df)

            def __getitem__(self, idx):
                text = self.df.iloc[idx, 1]
                label = int(self.df.iloc[idx, 0])
                return text, label


        ###########입력받은 데이터 처리 실행하는 메소드 showtell_classfy() ###############
        #result_all.html에서 입력받을 text를 contents에 넣고 전처리 후 데이터프레임에 넣어줌
        def showtell_classfy(text):
            contents = str(text)
            preprossed_contents_df = sentence_to_df(contents)

            preprossed_contents_df.dropna(inplace=True)
            #전처리된 데이터를 확인(데이터프레임으로 가공됨)
            preprossed_contents_df__ = preprossed_contents_df.sample(frac=1, random_state=999)
            

            #파이토치에 입력하기 위해서 로딩...
            ST_test_dataset = STDataset(preprossed_contents_df__)
            test_loader = DataLoader(ST_test_dataset, batch_size=1, shuffle=True, num_workers=0)
            #로딩되는지 확인
            ST_test_dataset.__getitem__(1)

            #time.sleep(1)



            #check whether cuda is available
            #device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')  
            device = torch.device("cpu")  
            #device = torch.device("cuda")
            #tokenizer = BertTokenizer.from_pretrained('bert-base-cased')
            tokenizer = BertTokenizer.from_pretrained('bert-large-cased')
            model = BertForSequenceClassification.from_pretrained('bert-large-cased')
            model.to(device)



            # for text, label in test_loader :
            #     print("text:",text)
            #     print("label:",label)


            #저장된 모델을 불러온다.
            #J:\Django\EssayFit_Django\essayfitaiproject\essayfitapp\model.pt
            #time.sleep(1)
            #model = torch.load("/Users/jongphilkim/Desktop/Django_WEB/essayfitaiproject/essayai/model.pt", map_location=torch.device('cpu'))
            model = torch.load("model.pt", map_location=torch.device('cpu'))
            print("model loadling~")
            model.eval()


            pred_loader = test_loader
            print("pred_loader:", pred_loader)
            total_loss = 0
            total_len = 0
            total_showing__ = 0
            total_telling__ = 0

            showing_conunter = [] #문장에 해당하는 SHOWING을 계산한다.
            
            print("check!")
            for text, label in pred_loader:
                print("text:",text)
                #print("label:",label)
                encoded_list = [tokenizer.encode(t, add_special_tokens=True) for t in text] #text to tokenize
                padded_list =  [e + [0] * (512-len(e)) for e in encoded_list] #padding
                sample = torch.tensor(padded_list) #torch tensor로 변환
                sample, label = sample.to(device), label.to(device) #tokenized text에 label을 넣어서 Device(gpu/cpu)에 넣기 위해 준비
                labels = torch.tensor(label) #레이블을 텐서로 변환
                #time.sleep(1)
                outputs = model(sample,labels=labels) #모델을 통해서 샘플텍스트와 레이블 입력데이터를 출력 output에 넣음
                #시간 딜레이를 주자
                #time.sleep(1)
                _, logits = outputs #outputs를 로짓에 넣음 이것을 softmax에 넣으면 0~1 사이로 결과가 출력됨
                
                pred = torch.argmax(F.softmax(logits), dim=1) #드디어 예측한다. argmax는 리스트(계산된 값)에서 가장 큰 값을 추출하여 pred에 넣는다. 0 ~1 사이의 값이 나올거임
                print('pred :', pred)
                # correct = pred.eq(labels) 
                showing__ = pred.eq(1) # 예측한 결과가 1과 같으면 showing이다   >> TRUE   SHOWING을 추출하려면 이것만 카운드하면 된다. 
                telling__ = pred.eq(0) # 예측한 결과가 0과 같으면 telling이다   >> FALSE
                
                #print('showing : ', showing__)
                #print('telling : ', telling__)
                
                
                showing_conunter.append(text)        
                #pred_ = round(float(pred))
                showing_conunter.append(pred)



            return showing_conunter 


        st_re = showtell_classfy(str(input_sent_chr)) # 캐릭터거 포함된 문장(전처리 완료된) 입력

        df = DataFrame(st_re)
        df_ = df[0::2] # 글만 추출
        df_label = df[1::2] # 레이블만 추출

        df_.reset_index(drop=True, inplace=True) #데이터를 합치기 위해서 초기화
        df_label.reset_index(drop=True, inplace=True)

        df_result = pd.concat([df_,df_label],axis=1) #합치기

        df_result.columns = ['sentence','show/tell']

        df_fin = df_result['show/tell'].value_counts(normalize=True)
        list(df_fin)
        showing_sentence_with_char = max(round(df_fin*100))

        # print("===============================================================")
        # print ('Character Descriptiveness : ', showing_sentence_with_char)
        # print("===============================================================")

        return showing_sentence_with_char



    ################################################
    #############  Emphasis on YOU  ################
    ################################################
    def EmphasisOnYou(text):

        essay_input_corpus = str(text) #문장입력
        essay_input_corpus = essay_input_corpus.lower()#소문자 변환

        sentences  = sent_tokenize(essay_input_corpus) #문장 토큰화
        total_sentences = len(sentences)#토큰으로 처리된 총 문장 수
        total_words = len(word_tokenize(essay_input_corpus))# 총 단어수
        split_sentences = []
        for sentence in sentences:
            processed = re.sub("[^a-zA-Z]"," ", sentence)
            words = processed.split()
            split_sentences.append(words)

        skip_gram = 1
        workers = multiprocessing.cpu_count()
        bigram_transformer = Phrases(split_sentences)

        model = gensim.models.word2vec.Word2Vec(bigram_transformer[split_sentences], workers=workers, sg=skip_gram, min_count=1)

        model.train(split_sentences, total_examples=sum([len(sentence) for sentence in sentences]), epochs=100)
        
        #모델 설계 완료

        #캐릭터 표현하는 단어들을 리스트에 넣어서 필터로 만들고
        character_list = ['i', 'I', 'my', 'me', 'mine', 'one']
        
        ####문장에 char_list의 단어들이 있는지 확인하고, 있다면 유사단어를 추출한다.
        #우선 토큰화한다.
        retokenize = RegexpTokenizer("[\w]+") #줄바꿈 제거하여 한줄로 만들고
        token_input_text = retokenize.tokenize(essay_input_corpus)
        #print (token_input_text) #토큰화 처리 확인.. 토큰들이 리스트에 담김
        #리트스로 정리된 개별 토큰을 char_list와 비교해서 존재하는 것만 추출한다.
        filtered_chr_text = []
        for k in token_input_text:
            for j in character_list:
                if k == j:
                    filtered_chr_text.append(j)
        
        #print (filtered_chr_text) # 유사단어 비교 추출 완료, 겹치는 단어는 제거하자.
        
        filtered_chr_text_ = set(filtered_chr_text) #중복제거
        filtered_chr_text__ = list(filtered_chr_text_) #다시 리스트로 변환
        print (filtered_chr_text__) # 중복값 제거 확인
        
        for i in filtered_chr_text__:
            ext_sim_words_key = model.most_similar_cosmul(i) #모델적용
        
        char_total_count = len(filtered_chr_text) # 중복이 제거되지 않은 에세이 총 문장에 사용된 캐릭터 표현 수
        char_count_ = len(filtered_chr_text__) #중복제거된 캐릭터 표현 총 수
            
        result_char_ratio = round(char_total_count/total_words * 100, 2)
        return char_total_count


    # EmphasisOnYou_ = EmphasisOnYou(input_text) # 문장에서 키워드와 관련된 단어을 모두 추출하면 이런 결과가 나옴, 이 결과를 모두 합쳐서 캐릭터 총 값 계산해서 숫자로 출력
    # EmphasisOnYou_
    # print ('=============================================')
    # print ('Emphasis on You :', EmphasisOnYou_)
    # print ('=============================================')



    #########################################
    ######### Emphasis on others  ###########
    #########################################
    def EmphasisOnOthers(text):

        essay_input_corpus = str(text) #문장입력
        essay_input_corpus = essay_input_corpus.lower()#소문자 변환

        sentences  = sent_tokenize(essay_input_corpus) #문장 토큰화
        total_sentences = len(sentences)#토큰으로 처리된 총 문장 수
        total_words = len(word_tokenize(essay_input_corpus))# 총 단어수
        split_sentences = []
        for sentence in sentences:
            processed = re.sub("[^a-zA-Z]"," ", sentence)
            words = processed.split()
            split_sentences.append(words)

        skip_gram = 1
        workers = multiprocessing.cpu_count()
        bigram_transformer = Phrases(split_sentences)

        model = gensim.models.word2vec.Word2Vec(bigram_transformer[split_sentences], workers=workers, sg=skip_gram, min_count=1)

        model.train(split_sentences, total_examples=sum([len(sentence) for sentence in sentences]), epochs=100)
        
        #모델 설계 완료

        #캐릭터 표현하는 단어들을 리스트에 넣어서 필터로 만들고
        character_list = ['you', 'your', 'they','them',
                        'yours', 'he','him','his' 'she','her','it','someone','their', 'myself', 'aunt',
                        'brother','cousin','daughter','father','grandchild','granddaughter','granddson','grandfather',
                        'grandmother','great-grandchild','husband','ex-husband','son-in-law', 'daughter-in-law','mother',
                        'niece','nephew','parents','sister','son','stepfather','stepmother','stepdaughter', 'stepson',
                        'twin','uncle','widow','widower','wife','ex-wife','aunt',
                        'baby', 'beget', 'brother', 'buddy', 'conserve', 'counterpart', 'cousin',
                        'daughter', 'don', 'duplicate', 'ex', 'father', 'forefather', 'founder', 'gemini',
                        'grandchild', 'granddaughter', 'grandfather', 'grandma', 'he', 'helium', 'husband',
                        'in', 'law', 'match', 'mother', 'nephew', 'niece', 'parent', 'person', 'rear',
                        'sister', 'son', 'stepdaughter', 'stepfather', 'stepmother', 'stepson', 'twin', 'uncle', 'widow',
                        'widower', 'wife']
        
        ####문장에 char_list의 단어들이 있는지 확인하고, 있다면 유사단어를 추출한다.
        #우선 토큰화한다.
        retokenize = RegexpTokenizer("[\w]+") #줄바꿈 제거하여 한줄로 만들고
        token_input_text = retokenize.tokenize(essay_input_corpus)
        #print (token_input_text) #토큰화 처리 확인.. 토큰들이 리스트에 담김
        #리트스로 정리된 개별 토큰을 char_list와 비교해서 존재하는 것만 추출한다.
        filtered_chr_text = []
        for k in token_input_text:
            for j in character_list:
                if k == j:
                    filtered_chr_text.append(j)
        
        #print (filtered_chr_text) # 유사단어 비교 추출 완료, 겹치는 단어는 제거하자.
        
        filtered_chr_text_ = set(filtered_chr_text) #중복제거
        filtered_chr_text__ = list(filtered_chr_text_) #다시 리스트로 변환
        print (filtered_chr_text__) # 중복값 제거 확인
        
        for i in filtered_chr_text__:
            ext_sim_words_key = model.most_similar_cosmul(i) #모델적용
        
        char_total_count = len(filtered_chr_text) # 중복이 제거되지 않은 에세이 총 문장에 사용된 캐릭터 표현 수
        char_count_ = len(filtered_chr_text__) #중복제거된 캐릭터 표현 총 수
            
        result_char_ratio = round(char_total_count/total_words * 100, 2)
    
    
        return char_total_count


    # EmphasisOnOthers_ = EmphasisOnOthers(input_text) # 문장에서 키워드와 관련된 단어을 모두 추출하면 이런 결과가 나옴, 이 결과를 모두 합쳐서 캐릭터 총 값 계산해서 숫자로 출력
    # EmphasisOnOthers_
    # print ('=============================================')
    # print ('Emphasis on Others :', EmphasisOnOthers_)
    # print ('=============================================')


    character_descriptiveness = character_descrip(text)
    print("===============================================================")
    print ('Character Descriptiveness : ' , character_descriptiveness)
    print("===============================================================")


    number_of_characters = NumberofCharacters(text) 
    print ('=============================================')
    print ('Number of Characters :' , number_of_characters)
    print ('=============================================')


    EmphasisOnYou_ = EmphasisOnYou(text)
    print ('=============================================')
    print ('Emphasis on You :' , EmphasisOnYou_)
    print ('=============================================')


    EmphasisOnOthers_ = EmphasisOnOthers(text) 
    print ('=============================================')
    print ('Emphasis on Others :' , EmphasisOnOthers_)
    print ('=============================================')


    return character_descriptiveness, number_of_characters, EmphasisOnYou_, EmphasisOnOthers_



In [19]:
#!/usr/bin/python
# -*- coding:utf-8 -*-

import json
import pandas as pd
import sys
import os
from tqdm import tqdm

path = "C:\\Users\\cacki\\Projects\\008_AI\\Story_Analysis-master8\\Story_Analysis-master\\CHARACTER_\\ps_data_test\\"
file_list = os.listdir(path)
file_list

['each_personal_essay_0.json',
 'each_personal_essay_1.json',
 'each_personal_essay_10.json',
 'each_personal_essay_11.json',
 'each_personal_essay_12.json',
 'each_personal_essay_13.json',
 'each_personal_essay_14.json',
 'each_personal_essay_15.json',
 'each_personal_essay_2.json',
 'each_personal_essay_21.json',
 'each_personal_essay_22.json',
 'each_personal_essay_23.json',
 'each_personal_essay_24.json',
 'each_personal_essay_25.json',
 'each_personal_essay_26.json',
 'each_personal_essay_27.json',
 'each_personal_essay_28.json',
 'each_personal_essay_29.json',
 'each_personal_essay_3.json',
 'each_personal_essay_30.json',
 'each_personal_essay_4.json',
 'each_personal_essay_5.json',
 'each_personal_essay_6.json',
 'each_personal_essay_7.json',
 'each_personal_essay_8.json',
 'each_personal_essay_9.json']

In [20]:
essay_results = []

for file_name_raw in tqdm(file_list): # 파일이름을 하나씩 불러와서
    file_name = path  + file_name_raw #경로를 추가하고
    with open(file_name, 'r') as json_file : #JSON파일로 파일을 열어서
        ps_input_each_data = json.load(json_file) # JSON파일로 로드한다.
        input_data = json.dumps(ps_input_each_data)
        re_each_ess = character_all_section(input_data)
        essay_results.append(re_each_ess)

  0%|          | 0/26 [00:00<?, ?it/s]

model loadling~
pred_loader: <torch.utils.data.dataloader.DataLoader object at 0x0000023841B4C788>
check!
text: ('I also follow many different fashion accounts on instagram and like to see what other people are wearing and their style.',)


C:\Users\cacki\anaconda3\envs\py37pytorch\lib\site-packages\ipykernel_launcher.py:210: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
C:\Users\cacki\anaconda3\envs\py37pytorch\lib\site-packages\ipykernel_launcher.py:217: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


pred : tensor([1])
text: ('After looking through the Columbia Spectator, I noticed an article about the native presence at Columbia, or rather lack thereof.',)
pred : tensor([1])
text: ('The only thing other than stubbornness that my parents have in common is their devotion to academics.',)
pred : tensor([1])
text: ('"}, {"index": 1, "personal_essay": "My ideal college is a combination of challenging and supportive.',)
pred : tensor([1])
text: ('I also need a community that will support me and my needs, allowing me to flourish in an otherwise challenging environment.',)
pred : tensor([1])
text: ("These racist comments highlight the battle I've been in for a lifetime.A white classmate recently questioned my authenticity as a Native, and another in the same week told me with envy that I was a 'check-box' for colleges.",)
pred : tensor([1])
text: ('My parents have been divorced for five years and have been at war ever since.',)
pred : tensor([0])
text: ('I hope to have challenging academi

pred : tensor([1])
text: ('I am currently reading Between the World and Me by Ta-Nehisi Coates.',)
pred : tensor([1])
text: ('(150 words or less)*List the titles of the print, electronic publications and websites you read regularly.',)
pred : tensor([1])
text: ('I also recently re-read Jane Eyre by Charlotte Bronte and The Perks of Being a Wallflower by Stephen Chbosky, my two favorite books ever.',)
pred : tensor([1])
text: ('I have also been enjoying documentaries such as The Mars Generation, Reversing Roe, Forks over Knives, and Take your Pills.',)
pred : tensor([1])
text: ("The article, written by a member of Columbia's cultural center, pointed out that the school should do a better job of recruiting native students from many different tribes, socio economic backgrounds, and regions.",)
pred : tensor([1])
text: ('If my Danish mom and Native dad cannot be together, how can those two aspects of my identity be together in me?In my life constantly moving between houses, it is hard to f

...

C:\Users\cacki\anaconda3\envs\py37pytorch\lib\site-packages\ipykernel_launcher.py:60: DeprecationWarning: Call to deprecated `most_similar_cosmul` (Method will be removed in 4.0.0, use self.wv.most_similar_cosmul() instead).


['i', 'my', 'me']
Emphasis on You : 114


C:\Users\cacki\anaconda3\envs\py37pytorch\lib\site-packages\ipykernel_launcher.py:310: DeprecationWarning: Call to deprecated `most_similar_cosmul` (Method will be removed in 4.0.0, use self.wv.most_similar_cosmul() instead).
C:\Users\cacki\anaconda3\envs\py37pytorch\lib\site-packages\ipykernel_launcher.py:387: DeprecationWarning: Call to deprecated `most_similar_cosmul` (Method will be removed in 4.0.0, use self.wv.most_similar_cosmul() instead).
 19%|█▉        | 5/26 [16:11<1:08:01, 194.34s/it]

['they', 'them', 'it', 'in', 'you', 'myself']
Emphasis on Others : 32


IndexError: single positional indexer is out-of-bounds

In [21]:
essay_results

[(88.0, 388, 185, 101),
 (84.0, 258, 122, 73),
 (91.0, 282, 125, 85),
 (14.0, 468, 198, 177),
 (85.0, 211, 114, 32)]

In [22]:
df_r = pd.DataFrame(essay_results, columns =['char_desc', 'num_chars', 'emph_you', 'emph_others'])  #e데이터프레임에 넣기
df_r

,char_desc,num_chars,emph_you,emph_others
0,88.0,388,185,101
1,84.0,258,122,73
2,91.0,282,125,85
3,14.0,468,198,177
4,85.0,211,114,32


In [23]:
#1000명 데이터의 각 값의 평균 계산
char_desc_mean = df_r['char_desc'].mean()
char_desc_mean

72.4

In [24]:
num_chars_mean = df_r['num_chars'].mean()
num_chars_mean

321.4

In [25]:
emph_you_mean = df_r['emph_you'].mean()
emph_you_mean 

148.8

In [26]:
emph_others_mean = df_r['emph_others'].mean()
emph_others_mean

93.6

In [27]:
# 30명의 최종 평균값 계산
char_re_30 = [char_desc_mean, num_chars_mean, emph_you_mean, emph_others_mean]
char_re_30

[72.4, 321.4, 148.8, 93.6]